# Named Entity Recognition on the CoNLL++ Dataset

Notebook to train a [flair](https://github.com/flairNLP/flair) model using stacked embeddings (with word and flair contextual embeddings) to perform named entity recognition (NER). The dataset used is the [CoNLL 2003](https://www.aclweb.org/anthology/W03-0419.pdf) dataset for NER (train, dev) with a manually corrected (improved/cleaned) test set from the [CrossWeigh paper](https://arxiv.org/abs/1909.01441) called [CoNLL++](https://github.com/ZihanWangKi/CrossWeigh#data). The current state-of-the-art model on this dataset is from the CrossWeigh paper (also using flair) by [Wang et al. (2019)](https://www.aclweb.org/anthology/D19-1519/) with F1-score of [94.3%](http://nlpprogress.com/english/named_entity_recognition.html). Without using pooled-embeddings, CrossWeigh and training to a max 50 instead of 150 epochs, we get a micro F1-score of 93.5%, within 0.7 of a percentage point of the SOTA.

The notebook is structured as follows:
* Setting up the GPU Environment
* Getting Data
* Training and Testing the Model
* Using the Model (Running Inference)

## Task Description

> Named entity recognition (NER) is the task of tagging entities in text with their corresponding type. Approaches typically use BIO notation, which differentiates the beginning (B) and the inside (I) of entities. O is used for non-entity tokens.

#### Install Dependencies

In [ ]:
%%capture
! pip install -q flair

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import urllib.request
from pathlib import Path
import shutil

import flair
from typing import List
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings, TransformerWordEmbeddings
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# Getting Data

We download the data (train, test and dev sets) in the BIO format (each token in each of the sentences are tagged with Begin, Inside or Outside labels) as text files from the [CoNLL++ repository](https://github.com/ZihanWangKi/CrossWeigh) and save them to the `/content/data/` folder.

In [ ]:
def download_file(url, output_file):
  Path(output_file).parent.mkdir(parents=True, exist_ok=True)
  urllib.request.urlretrieve (url, output_file)

download_file('https://raw.githubusercontent.com/ZihanWangKi/CrossWeigh/master/data/conllpp_train.txt', '/content/data/conllpp_train.txt')
download_file('https://raw.githubusercontent.com/ZihanWangKi/CrossWeigh/master/data/conllpp_dev.txt', '/content/data/conllpp_dev.txt')
download_file('https://raw.githubusercontent.com/ZihanWangKi/CrossWeigh/master/data/conllpp_test.txt', '/content/data/conllpp_test.txt')

Now we will use flair's built in `ColumnCorpus` object to load in our `conllpp_train.txt`, `conllpp_test.txt` and `conllpp_dev.txt` files in the `/content/data/` folder.

In [ ]:
columns = {0: 'text', 3: 'ner'}
corpus: Corpus = ColumnCorpus('/content/data/', columns,
                              train_file='conllpp_train.txt',
                              test_file='conllpp_test.txt',
                              dev_file='conllpp_dev.txt')

2022-06-17 08:35:59,930 Reading data from /content/data
2022-06-17 08:35:59,934 Train: /content/data/conllpp_train.txt
2022-06-17 08:35:59,937 Dev: /content/data/conllpp_dev.txt
2022-06-17 08:35:59,939 Test: /content/data/conllpp_test.txt


To check that the sentences/size of the train, test and development set tally exactly with the Table 1 (train: 14987, test: 3684, dev: 3466) in the [CoNLL 2003 paper](https://www.aclweb.org/anthology/W03-0419.pdf), we will get the `len()` of the `.train`, `.test` and `.dev` sets from the `ColumnCorpus` object and print it out as a table.

In [ ]:
import pandas as pd
data = [[len(corpus.train), len(corpus.test), len(corpus.dev)]]
# Prints out the dataset sizes of train test and development in a table.
pd.DataFrame(data, columns=["Train", "Test", "Development"])

,Train,Test,Development
0,14987,3684,3466


# Training and Testing the Model

#### Train the Model

To train the flair `SequenceTagger`, we use the `ModelTrainer` object with the corpus and the tagger to be trained. We use flair's sensible default options while specifying the output folder for the `SequenceTagger` model to be `/content/model/conllpp`. We also set the `embeddings_storage_mode` to be `gpu` to utilise the GPU.

In [ ]:
%%capture
tag_type = 'ner'
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)
# For faster training and smaller models, we can comment out the flair embeddings.
# This will significantly affect the performance though.
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
    TransformerWordEmbeddings('bert-base-uncased')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=False)

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

2022-06-17 08:36:07,292 Computing label dictionary. Progress:
2022-06-17 08:36:07,757 Dictionary created for label 'ner' with 5 values: LOC (seen 7140 times), PER (seen 6600 times), ORG (seen 6321 times), MISC (seen 3438 times)
2022-06-17 08:36:08,264 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmph49_am7f
2022-06-17 08:36:15,836 copying /tmp/tmph49_am7f to cache at /root/.flair/embeddings/glove.gensim.vectors.npy
2022-06-17 08:36:16,038 removing temp file /tmp/tmph49_am7f
2022-06-17 08:36:16,401 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpjg2stgyr
2022-06-17 08:36:18,503 copying /tmp/tmpjg2stgyr to cache at /root/.flair/embeddings/glove.gensim
2022-06-17 08:36:18,533 removing temp file /tmp/tmpjg2stgyr
2022-06-17 08:36:20,731 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in 

In [ ]:
trainer.train('/content/model/conllpp',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50,
              embeddings_storage_mode='cpu')

2022-06-17 08:37:01,559 ----------------------------------------------------------------------------------------------------
2022-06-17 08:37:01,562 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_3): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Em

100%|██████████| 109/109 [00:37<00:00,  2.92it/s]

2022-06-17 08:41:27,724 Evaluating as a multi-label problem: False
2022-06-17 08:41:27,782 DEV : loss 0.11812306940555573 - f1-score (micro avg)  0.869


2022-06-17 08:41:28,261 BAD EPOCHS (no improvement): 0
2022-06-17 08:41:28,266 saving best model
2022-06-17 08:41:30,583 ----------------------------------------------------------------------------------------------------
2022-06-17 08:41:50,262 epoch 2 - iter 46/469 - loss 0.10664816 - samples/sec: 74.84 - lr: 0.100000
2022-06-17 08:42:09,272 epoch 2 - iter 92/469 - loss 0.10926090 - samples/sec: 77.47 - lr: 0.100000
2022-06-17 08:42:27,729 epoch 2 - iter 138/469 - loss 0.10664468 - samples/sec: 79.79 - lr: 0.100000
2022-06-17 08:42:46,584 epoch 2 - iter 184/469 - loss 0.10548982 - samples/sec: 78.10 - lr: 0.100000
2022-06-17 08:43:05,659 epoch 2 - iter 230/469 - loss 0.10270659 - samples/sec: 77.20 - lr: 0.100000
2022-06-17 08:43:23,786 epoch 2 - iter 276/469 - loss 0.10215463 - samples/sec: 81.24 - lr: 0.100000
2022-06-17 08:43:42,378 epoch 2 - iter 322/469 - loss 0.10103467 - samples/sec: 79.21 - lr: 0.100000
2022-06-17 08:44:00,867 epoch 2 - iter 368/469 - loss 0.09971832 - sample

100%|██████████| 109/109 [00:24<00:00,  4.52it/s]

...

2022-06-17 11:37:54,199 Evaluating as a multi-label problem: False
2022-06-17 11:37:54,259 DEV : loss 0.16759750247001648 - f1-score (micro avg)  0.9438


2022-06-17 11:37:54,728 BAD EPOCHS (no improvement): 1
2022-06-17 11:37:57,015 ----------------------------------------------------------------------------------------------------
2022-06-17 11:37:57,019 loading file /content/model/conllpp/best-model.pt
2022-06-17 11:38:00,649 SequenceTagger predicts: Dictionary with 17 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC


100%|██████████| 116/116 [00:26<00:00,  4.38it/s]

2022-06-17 11:38:27,796 Evaluating as a multi-label problem: False
2022-06-17 11:38:27,851 0.9167	0.9149	0.9158	0.8764
2022-06-17 11:38:27,853 
Results:
- F-score (micro) 0.9158
- F-score (macro) 0.9026
- Accuracy 0.8764

By class:
              precision    recall  f1-score   support

         ORG     0.8954    0.8682    0.8816      1715
         LOC     0.9296    0.9544    0.9418      1646
         PER     0.9726    0.9654    0.9690      1618
        MISC     0.8128    0.8230    0.8179       723

   micro avg     0.9167    0.9149    0.9158      5702
   macro avg     0.9026    0.9028    0.9026      5702
weighted avg     0.9167    0.9149    0.9157      5702

2022-06-17 11:38:27,854 ----------------------------------------------------------------------------------------------------


{'dev_loss_history': [0.11812306940555573,
  0.06859636306762695,
  0.06514368206262589,
  0.05592651292681694,
  0.06525728851556778,
  0.0605003647506237,
  0.07066928595304489,
  0.061316538602113724,
  0.06465733796358109,
  0.08457881957292557,
  0.06976239383220673,
  0.09573764353990555,
  0.08056985586881638,
  0.10226253420114517,
  0.10129936039447784,
  0.10863196849822998,
  0.09911255538463593,
  0.12595795094966888,
  0.12453508377075195,
  0.125911146402359,
  0.1391693353652954,
  0.1427735984325409,
  0.1388322114944458,
  0.14742818474769592,
  0.15061542391777039,
  0.1478821039199829,
  0.15215998888015747,
  0.1583978235721588,
  0.15556968748569489,
  0.16040655970573425,
  0.1576801836490631,
  0.1598232239484787,
  0.16121862828731537,
  0.16081681847572327,
  0.16388776898384094,
  0.16377849876880646,
  0.16497555375099182,
  0.16435116529464722,
  0.1650353968143463,
  0.16567803919315338,
  0.16575579345226288,
  0.1660126894712448,
  0.16627006232738495,
  

We see that the output accuracy (F1-score) for our new model is **93.5%** (F1-score (micro) 0.9354).

## Using the Model (Running Inference)

Running the model to do some predictions/inference is as simple as calling `tagger.predict(sentence)`.

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

input_sentence = [
                  'George Washington went to Washington to study in University of Washington.',
                  'Kobe Bryant is top 10 players in NBA history.',
                  'Stephen Curry has 4 NBA titles.',
                  'President Bush is hiding in bushes in Irap.',
                  'Steve Jobs is find a job.'
                  ]
tagger: SequenceTagger = SequenceTagger.load("/content/model/conllpp/final-model.pt")

for sentence in input_sentence:    
    sentence_each: Sentence = Sentence(sentence)
    tagger.predict(sentence_each)
    print(sentence_each.to_tagged_string())

2022-06-17 08:20:27,984 loading file /content/model/conllpp/final-model.pt
2022-06-17 08:20:28,716 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-MISC, B-MISC, E-MISC, I-MISC, <START>, <STOP>
Sentence: "George Washington went to Washington to study in University of Washington ." → ["George Washington"/PER, "Washington"/LOC, "University"/LOC, "Washington"/LOC]
Sentence: "Kobe Bryant is top 10 players in NBA history ." → ["Kobe Bryant"/PER]
Sentence: "Stephen Curry has 4 NBA titles ." → ["Stephen Curry"/PER]
Sentence: "President Bush is hiding in bushes in Irap ." → ["Bush"/LOC, "Irap"/LOC]


Move the model files from our local directory to your Google Drive.

In [ ]:
shutil.move('/content/model/conllpp/', "/content/drive/My Drive/model/")